In [2]:
import math
import random
import itertools
import matplotlib.pyplot as plt
from tqdm import tqdm

import numpy as np
import numba
import numpy.random
from collections import deque

import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, Reshape
from tensorflow.keras.optimizers import Adam

In [7]:
class LineWorld:
    def __init__(self, nb_cells, start_cell, good_end_cell, bad_end_cell):
        self.done = None
        self.current_state = start_cell  # État actuel
        self.end_good_state = good_end_cell # État final
        self.end_bad_state = bad_end_cell
        self.reward = 0.0
        self.num_states = nb_cells  # Nombre total d'états
        self.states = [i for i in range(nb_cells)]
        self.actions = [0, 1]
        self.num_actions = 2  # Nombre total d'actions possibles
        self.line_world = ["_"] * (self.num_states - 1)
        self.line_world.insert(self.current_state,"X")
        

    def reset(self):
        self.reward = 0.0
        self.done = False
        self.current_state = np.random.randint(1,8)
        
    def isTerminateState(self):
        if (self.current_state == self.end_good_state) or (self.current_state == self.end_bad_state):
            return True
        else:
            return False
        
    def step(self, action):
        # Si l'action est 1, on avance à droite
        if (action == 1):
            self.current_state += 1
            self.reward = 0  # Pas de récompense pour avancer
            self.line_world.remove("X")
            self.line_world.insert(self.current_state,"X")
            print(self.line_world)
        # Si l'action est 0, on avance à gauche
        elif (action == 0):
            self.current_state-= 1
            self.reward = 0  # Pas de récompense pour avancer
            self.line_world.remove("X")
            self.line_world.insert(self.current_state, "X")
            print(self.line_world)
        # Si l'on atteint l'état final, la partie est terminée
        if self.current_state == self.end_good_state:
            self.reward = 1  # Récompense de 1 pour atteindre l'état final
            #print(self.line_world)
            self.done = True
        elif self.current_state == self.end_bad_state:
            self.reward = -1
            #print(self.line_world)
            self.done = True
        return self.current_state, self.reward, self.done
        
            

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.random
import tensorflow as tf
from tqdm import tqdm


def REINFORCE(env: GridWorld, max_iter_count: int = 10000,
              gamma: float = 0.99,
              alpha: float = 0.1):
    pi = tf.keras.models.Sequential()
    pi.add(tf.keras.layers.Dense(env.num_actions,
                                 activation=tf.keras.activations.softmax,
                                 use_bias=True
                                 ))

    ema_score = 0.0
    ema_nb_steps = 0.0
    first_episode = True

    step = 0
    ema_score_progress = []
    ema_nb_steps_progress = []

    episode_states_buffer = []
    episode_actions_buffer = []
    episode_rewards_buffer = []

    for _ in tqdm(range(max_iter_count)):
        if env.done:
            ### TRAINING TIME !!!
            G = 0.0

            for t in reversed(range(0, len(episode_states_buffer))):
                G = episode_rewards_buffer[t] + gamma * G

                with tf.GradientTape() as tape:
                    pi_s_a_t = pi(np.array([episode_states_buffer[t]]))[0][episode_actions_buffer[t]]
                    log_pi_s_a_t = tf.math.log(pi_s_a_t)

                grads = tape.gradient(log_pi_s_a_t, pi.trainable_variables)

                for (v, g) in zip(pi.trainable_variables, grads):
                    if g is not None:
                        v.assign_add(alpha * (gamma ** t) * G * g)

            if first_episode:
                ema_score = env.reward
                ema_nb_steps = step
                first_episode = False
            else:
                ema_score = (1 - 0.9) * env.reward + 0.9 * ema_score
                ema_nb_steps = (1 - 0.9) * step + 0.9 * ema_nb_steps
                ema_score_progress.append(ema_score)
                ema_nb_steps_progress.append(ema_nb_steps)

            env.reset()
            episode_states_buffer.clear()
            episode_actions_buffer.clear()
            episode_rewards_buffer.clear()
            step = 0

        s = np.array([env.currentIntState])

        episode_states_buffer.append(s)

        aa = env.actions

        pi_s = pi(np.array([s]))[0].numpy()
        allowed_pi_s = pi_s[aa]
        sum_allowed_pi_s = np.sum(allowed_pi_s)
        if sum_allowed_pi_s == 0.0:
            probs = np.ones((len(aa),)) * 1.0 / (len(aa))
        else:
            probs = allowed_pi_s / sum_allowed_pi_s
        print("s:  ",s)
        print("pi_s:  ", pi_s)
        print("allowed_pi_s:   ", allowed_pi_s)
        print("sum_allowed_pi_s:  ", sum_allowed_pi_s)
        print("probs:   ", probs)
        a = np.random.choice(aa, p=probs)

        episode_actions_buffer.append(a)

        old_score = env.reward
        env.step(a)
        new_score = env.reward
        r = new_score - old_score

        episode_rewards_buffer.append(r)

        step += 1
    return pi, ema_score_progress, ema_nb_steps_progress


game = GridWorld(taille=[6, 6], position_start=[0, 1], good_end_position=[4, 2], bad_end_position=[3, 5])
pi, scores, steps = REINFORCE(game, max_iter_count=10000)
print(pi.weights)
print(scores)

  0%|                                                                                | 6/10000 [00:00<03:26, 48.31it/s]

['_', '_', 'X', '_', '_', '_', '_', '_', '_', '_']
['_', '_', '_', 'X', '_', '_', '_', '_', '_', '_']
['_', '_', 'X', '_', '_', '_', '_', '_', '_', '_']
['_', 'X', '_', '_', '_', '_', '_', '_', '_', '_']
['X', '_', '_', '_', '_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_', 'X', '_', '_', '_']
['_', '_', '_', '_', '_', '_', '_', 'X', '_', '_']
['_', '_', '_', '_', '_', '_', '_', '_', 'X', '_']
['_', '_', '_', '_', '_', '_', '_', '_', '_', 'X']
['_', '_', '_', '_', '_', '_', 'X', '_', '_', '_']
['_', '_', '_', '_', '_', '_', '_', 'X', '_', '_']
['_', '_', '_', '_', '_', '_', '_', '_', 'X', '_']
['_', '_', '_', '_', '_', '_', '_', '_', '_', 'X']


  0%|▏                                                                              | 21/10000 [00:00<03:01, 55.10it/s]

['_', '_', '_', 'X', '_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', 'X', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '_', 'X', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_', 'X', '_', '_', '_']
['_', '_', '_', '_', '_', '_', '_', 'X', '_', '_']
['_', '_', '_', '_', '_', '_', '_', '_', 'X', '_']
['_', '_', '_', '_', '_', '_', '_', '_', '_', 'X']
['_', '_', '_', 'X', '_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', 'X', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '_', 'X', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_', 'X', '_', '_', '_']
['_', '_', '_', '_', '_', '_', '_', 'X', '_', '_']
['_', '_', '_', '_', '_', '_', '_', '_', 'X', '_']
['_', '_', '_', '_', '_', '_', '_', '_', '_', 'X']


  0%|▎                                                                              | 35/10000 [00:00<03:08, 52.82it/s]

...


['_', '_', '_', '_', '_', '_', '_', '_', '_', 'X']
['_', '_', '_', '_', '_', 'X', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_', 'X', '_', '_', '_']
['_', '_', '_', '_', '_', '_', '_', 'X', '_', '_']
['_', '_', '_', '_', '_', '_', '_', '_', 'X', '_']
['_', '_', '_', '_', '_', '_', '_', '_', '_', 'X']
['_', '_', '_', '_', '_', 'X', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_', 'X', '_', '_', '_']
['_', '_', '_', '_', '_', '_', '_', 'X', '_', '_']
['_', '_', '_', '_', '_', '_', '_', '_', 'X', '_']
['_', '_', '_', '_', '_', '_', '_', '_', '_', 'X']


 80%|█████████████████████████████████████████████████████████████▋               | 8015/10000 [02:21<00:34, 57.40it/s]

['_', '_', '_', 'X', '_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', 'X', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '_', 'X', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_', 'X', '_', '_', '_']
['_', '_', '_', '_', '_', '_', '_', 'X', '_', '_']
['_', '_', '_', '_', '_', '_', '_', '_', 'X', '_']
['_', '_', '_', '_', '_', '_', '_', '_', '_', 'X']
['_', '_', '_', 'X', '_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', 'X', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '_', 'X', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_', 'X', '_', '_', '_']
['_', '_', '_', '_', '_', '_', '_', 'X', '_', '_']
['_', '_', '_', '_', '_', '_', '_', '_', 'X', '_']
['_', '_', '_', '_', '_', '_', '_', '_', '_', 'X']


 80%|█████████████████████████████████████████████████████████████▊               | 8029/10000 [02:22<00:34, 57.71it/s]

['_', '_', '_', '_', '_', '_', '_', '_', 'X', '_']
['_', '_', '_', '_', '_', '_', '_', '_', '_', 'X']
['_', '_', '_', '_', '_', 'X', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_', 'X', '_', '_', '_']
['_', '_', '_', '_', '_', '_', '_', 'X', '_', '_']
['_', '_', '_', '_', '_', '_', '_', '_', 'X', '_']
['_', '_', '_', '_', '_', '_', '_', '_', '_', 'X']
['_', '_', '_', '_', '_', 'X', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_', 'X', '_', '_', '_']
['_', '_', '_', '_', '_', '_', '_', 'X', '_', '_']
['_', '_', '_', '_', '_', '_', '_', '_', 'X', '_']
['_', '_', '_', '_', '_', '_', '_', '_', '_', 'X']
